<h2>Create the Interactive Web App</h2>

Maps are one of the effective ways to display elements that have a given location. Traditional depictions typically use static maps presented in printed or digital forms. Newer depictions utilize specialized software capable of showing the map with interactive elements viewed with a computer or smartphones. Some of the strengths in producing an interactive map is that it can improve the user experience significantly by providing a layer for users to explore themselves. 

In this part of the assessment, we will create an interactive map with the package <code>folium</code>.

First, we have to import the GeoJSONs that represent the property and the classes.

In [1]:
import geopandas as gpd
import os

property_boundaries = gpd.read_file(os.path.join('geojson', 'property_boundaries.geojson'))
class_gdf = gpd.read_file(os.path.join('geojson', 'class.geojson'))

Next, we will use <code>folium</code> to design the interactive web map. Although <code>geopandas</code> has support for interactive maps (through <code>folium</code>), using <code>folium</code> provides better support for custom HTML/CSS.

In [2]:
import folium
from branca.element import Figure, Template, MacroElement

# Create map
m = folium.Map([-32.57, 149.05], zoom_start=14)

# Define a styling for the Slope Classes
def class_style(feature):
    slope_class = feature['properties']['class']
    return {
        'weight':0,
        'fillOpacity': 0.8,
        'fillColor':
            '#4daf4a' if slope_class == 1 else '#377eb8'
    }

# Add the slope class in the 
f = folium.GeoJson(
    class_gdf,
    name = "Slope Class",
    style_function=class_style
).add_to(m)

# Define a styling for the property boundaries
def property_style(feature):
    slope_class = feature['properties']['planlabel']
    return {
        'weight':3,
        'fillOpacity': 0,
        'color': 'black',
    }

# Load the property boundaries
g = folium.GeoJson(
    property_boundaries,
    style_function = property_style,
    tooltip = folium.features.GeoJsonTooltip(
        fields = ['planlabel'],
        aliases = ['Field ID'],
        labels = True,
        style="""
            font-size: 13px;
        """
    ), name = 'Client Property'
).add_to(m)

# Add ESRI Basemap as a basemap
# For demonstration purposes only.
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Basemap Powered by Esri.',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

folium.LayerControl().add_to(m)

## Legend
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 285px;
    right: 15px;
    width: 150px;
    height: 90px;
    z-index:9999;
    font-size:14px;
    border: 1px solid grey;
    padding: 10px 10px 10px 10px;
    background-color: #FFFFFF;
">
<h3><b>Legend</b></h3>
<ul style="list-style-type:none; margin-left:-0px; font-size: 14px;">
<li> <i class="fa fa-square fa-1x" style="color:#4daf4a;"></i> Slope < 15° </li>
<li> <i class="fa fa-square fa-1x" style="color:#377eb8;"></i> Slope ≥ 15° </li>
</ul>
</div>

<div style="
    position: fixed;
    bottom: 25px;
    right: 15px;
    width: 400px;
    z-index:9999;

">
<style>
.table_component {
    overflow: auto;
    width: 100%;
}

.table_component table {
    border: 1px solid #dededf;
    height: 100%;
    width: 100%;
    table-layout: fixed;
    border-collapse: collapse;
    border-spacing: 1px;
    text-align: center;
}

.table_component caption {
    caption-side: top;
    text-align: left;
}

.table_component th {
    border: 1px solid #dededf;
    background-color: #eceff1;
    color: #000000;
    padding: 5px;
}

.table_component td {
    border: 1px solid #dededf;
    background-color: #ffffff;
    color: #000000;
    padding: 5px;
}
</style>
<div class="table_component" role="region" tabindex="0">
<table>
    <caption><br></caption>
    <thead>
        <tr>
            <th>Field ID<br></th>
            <th>
                <div>
                    <div>Slope &lt; 15°</div>
                </div>
            </th>
            <th>
                <div>
                    <div>Slope ≥ 15°</div>
                </div>
            </th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>DP835442</td>
            <td>88.99%</td>
            <td>11.01%</td>
        </tr>
        <tr>
            <td>DP784596</td>
            <td>47.58%</td>
            <td>52.42%</td>
        </tr>
        <tr>
            <td>DP827602</td>
            <td>32.01%</td>
            <td>67.99%</td>
        </tr>
        <tr>
            <td>DP750779</td>
            <td>26.99%</td>
            <td>73.01%</td>
        </tr>
        <tr>
            <td>DP875823</td>
            <td>39.80%</td>
            <td>60.20%</td>
        </tr>
        <tr>
            <td>DP1213407</td>
            <td>78.44%</td>
            <td>21.56%</td>
        </tr>
        <tr>
            <td>DP827602</td>
            <td>37.26%</td>
            <td>62.74%<br></td>
        </tr>
    </tbody>
</table>
</div>

{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)


Lastly, we will export the map to HTML and load it a website. This can be directly viewed using a modern browser.

In [3]:
m.save('index.html')